In [2]:
from kiteconnect import KiteConnect
from datetime import datetime,date,timedelta 
from pymongo import MongoClient
import pandas as pd
import Config
import time
client = MongoClient(Config.DB_Hostname, Config.DB_Port)

kite = KiteConnect(api_key="f6y99004f52fcntd")
kite_session = kite.generate_session("YlzgXhKimitWExHlaMulgTVedH6bwMgF", api_secret="wkut36radrfk28vr2to4wmqhpv67akxv")
kite.set_access_token(kite_session["access_token"])

start_date=date(2024,2,21)
date_formatted=start_date.strftime("%Y-%m-%d")

while date_formatted!="2024-02-24":
    
    time.sleep(0.35)
    
    try:
        df=pd.DataFrame(kite.historical_data(256265, f"{date_formatted} 09:14:00", f"{date_formatted} 15:31:00", "minute"))
    except Exception as e:
        print("Not able to call KITE Server. Retrying !!! ",e)
        time.sleep(60)
        continue
        
    if len(df)==0:
        print(f"\nEmpty File for{date_formatted}\n")
    elif len(df)!=375:
        print(f"{date_formatted} has {len(df)} rows. Check manually")
        while len(df)<375:
            df.loc[len(df)] = list(df.loc[len(df)-1])
    else:
        df.sort_values(by=["date"],inplace=True)
        df["batch_id"]=list(range(1,376))
        df["date"]=date_formatted
        df.drop(columns=['volume'],inplace=True)
        print(client.Strategy.NIFTYOHLC.delete_many({"date":date_formatted}).deleted_count,f" documents deleted for {date_formatted}")
        print(len(client.Strategy.NIFTYOHLC.insert_many(df.to_dict('records')).inserted_ids),f" documents entered for {date_formatted}\n")
    
    
    start_date+=timedelta(days = 1)
    date_formatted=start_date.strftime("%Y-%m-%d")

375  documents deleted for 2024-02-21
375  documents entered for 2024-02-21

375  documents deleted for 2024-02-22
375  documents entered for 2024-02-22

0  documents deleted for 2024-02-23
375  documents entered for 2024-02-23

